# Setting up Langtrace with Mistral

This Notebook shows the instructions for setting up OpenTelemetry based tracing for Mistral with Langtrace AI.

The Goal for this notebook to showcase a simple RAG app where you can chat with the United states consititution pdf.

In [1]:
%pip install -r requirements.txt

  Obtaining dependency information for mistralai from https://files.pythonhosted.org/packages/8c/d7/629cdb94c53e37ac2c4d5955ca1555ea675af339baca8aff18e54a7998ec/mistralai-1.2.3-py3-none-any.whl.metadata
  Using cached mistralai-1.2.3-py3-none-any.whl.metadata (26 kB)
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/b9/87/d9a71b915b854fb2f343fd69aa977a531bb66ee01cddaa5e06a0edcdb97b/langchain-0.3.8-py3-none-any.whl.metadata
  Using cached langchain-0.3.8-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/ee/09/fe45363955df325fb0aba3757b6b5bb0c530b6de257481c445ba39492dbf/langchain_community-0.3.8-py3-none-any.whl.metadata
  Using cached langchain_community-0.3.8-py3-none-any.whl.metadata (2.9 kB)
  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/5f/7a/10bf5dc92d13cc03230190fcc5016a0b138d99e5b36b8b89ee0fe1680e10/chro

## Imports & Initialize clients


In [4]:

import os
import chromadb
from mistralai import Mistral
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from langtrace_python_sdk import langtrace, with_langtrace_root_span


langtrace.init(api_key=os.environ["LANGTRACE_API_KEY"])
mistral = Mistral(api_key=os.environ["MISTRAL_API_KEY"])
client = chromadb.Client()



Initializing Langtrace SDK..
⭐ Leave our github a star to stay on top of our updates - https://github.com/Scale3-Labs/langtrace
Exporting spans to Langtrace cloud..


/Users/ali/Work/mistral-cookbook/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Use Langchain to split pdf into chunks

In [5]:
def load_and_split_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(data)
    return chunks

## Setup Chroma & Insert pdf chunks
Create a chroma collection, specifying the default embedding function which will be used in our RAG when inserting pdf chunks

In [6]:
def setup_chroma():

    return client.get_or_create_collection(
        name="mistral-rag",
        embedding_function=embedding_functions.DefaultEmbeddingFunction(),
    )


def add_documents_to_collection(collection, chunks):
    for i, chunk in enumerate(chunks):
        collection.add(
            documents=[chunk.page_content],
            metadatas=[{"source": chunk.metadata["source"]}],
            ids=[str(i)],
        )

## Query Collection
1. take query from user, get nearest 3 results from chunked pdf
2. construct a prompt structure
3. Give query and prompt to mistral for the actual response

In [7]:
def query_pdf(collection, query):
    results = collection.query(query_texts=[query], n_results=3)
    # Construct the prompt with context
    context = "\n".join(results["documents"][0])
    prompt = f"""Use the following context to answer the question. If you cannot answer based on the context, say so.

            Context:
            {context}

            Question: {query}

            Answer:"""
    response = mistral.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

## Run everything together and monitor using Langtrace.

In [9]:
@with_langtrace_root_span("main")
def main():
    print("Creating collection")
    collection = setup_chroma()
    print("Loading and splitting pdf")
    chunks = load_and_split_pdf("./constitution.pdf")
    print("Adding documents to collection")
    add_documents_to_collection(collection, chunks)
    print("Querying pdf")
    print(query_pdf(collection, "What is the purpose of the constitution?"))


if __name__ == "__main__":
    main()

That's it! Now you should be able to see the traces for all your inference calls on Langtrace!

## First Two Screenshots showcase the Trace and span structure of the whole RAG App.


![Trace 1](./TraceTree1.png)

![Trace 2](./TraceTree2.png)


## Second Two Screenshots are details of Mistral's Run.

- You can see what prompt is specfically fetched from chromadb and sent to mistral as well as the response

![Trace 1](./mistral-langtrace1.png)

![Trace 2](./mistral-langtrace2.png)